# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
csv_file = os.path.join('output_data', 'cities.csv')

city_df = pd.read_csv(csv_file)
city_df


,City,lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Atuona,-9.8000,-139.0333,78.10,69,23,15.43,PF,1635413937
1,Butaritari,3.0707,172.7902,81.30,73,100,4.99,KI,1635413724
2,Pasni,25.2631,63.4710,80.20,57,0,10.51,PK,1635413889
3,Mataura,-46.1927,168.8643,54.41,82,94,2.53,NZ,1635413691
4,Nanortalik,60.1432,-45.2371,33.84,77,76,6.96,GL,1635413795
...,...,...,...,...,...,...,...,...,...
579,Mbandaka,0.0487,18.2603,85.10,59,90,1.83,CD,1635414116
580,Brits,-25.6347,27.7802,72.82,48,99,9.93,ZA,1635414117
581,Breyten,-26.3018,29.9870,66.36,50,99,5.93,ZA,1635414117
582,Creel,27.7500,-107.6333,46.04,55,0,4.05,MX,1635414117


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
gmaps.configure(api_key=g_key)

locations = city_df[["lat", "Lng"]].astype(float)
humidity = city_df["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:
ideal_df = city_df[(city_df['Max Temp'] >= 65) & (city_df['Max Temp'] <= 80)]
ideal_df = ideal_df[ideal_df['Wind Speed'] <= 10]
ideal_df = ideal_df[ideal_df['Cloudiness'] <= 10]
ideal_df = ideal_df[ideal_df['Humidity'] <= 60]
ideal_df

,City,lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
160,Aljezur,37.3191,-8.8033,69.26,55,0,4.92,PT,1635413982
173,Nara,34.6851,135.8049,67.28,44,0,5.39,JP,1635413987
176,Guerrero Negro,27.9769,-114.0611,66.33,57,0,1.32,MX,1635413988
288,Sayyan,15.1718,44.3244,67.26,17,2,9.44,YE,1635414025
291,Ginda,30.6945,78.4932,65.12,38,0,4.74,IN,1635413897
313,Turaif,31.6725,38.6637,79.05,27,0,6.91,SA,1635414033
314,Neuquén,-38.9516,-68.0591,66.09,34,0,6.91,AR,1635414034
388,Fasa,28.9383,53.6482,76.24,11,0,0.00,IR,1635414056
404,San Miguel de Tucumán,-26.8241,-65.2226,66.20,52,0,8.05,AR,1635414060
429,Podgorica,42.4411,19.2636,73.51,34,0,1.14,ME,1635414069


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:
hotel_df = ideal_df
hotel_df['Hotel Name'] = ""
hotel_df

,City,lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
160,Aljezur,37.3191,-8.8033,69.26,55,0,4.92,PT,1635413982,
173,Nara,34.6851,135.8049,67.28,44,0,5.39,JP,1635413987,
176,Guerrero Negro,27.9769,-114.0611,66.33,57,0,1.32,MX,1635413988,
288,Sayyan,15.1718,44.3244,67.26,17,2,9.44,YE,1635414025,
291,Ginda,30.6945,78.4932,65.12,38,0,4.74,IN,1635413897,
313,Turaif,31.6725,38.6637,79.05,27,0,6.91,SA,1635414033,
314,Neuquén,-38.9516,-68.0591,66.09,34,0,6.91,AR,1635414034,
388,Fasa,28.9383,53.6482,76.24,11,0,0.00,IR,1635414056,
404,San Miguel de Tucumán,-26.8241,-65.2226,66.20,52,0,8.05,AR,1635414060,
429,Podgorica,42.4411,19.2636,73.51,34,0,1.14,ME,1635414069,


In [20]:
for index, row in hotel_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }


        lat = row['lat']
        lng = row['Lng']

        params['location'] = f"{lat}, {lng}"

        hotel_data = requests.get(base_url, params=params).json()
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
hotel_df

,City,lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
160,Aljezur,37.3191,-8.8033,69.26,55,0,4.92,PT,1635413982,Vicentina Hotel
173,Nara,34.6851,135.8049,67.28,44,0,5.39,JP,1635413987,Super Hotel Lohas JR Nara Eki
176,Guerrero Negro,27.9769,-114.0611,66.33,57,0,1.32,MX,1635413988,The Halfway Inn
288,Sayyan,15.1718,44.3244,67.26,17,2,9.44,YE,1635414025,NaN
291,Ginda,30.6945,78.4932,65.12,38,0,4.74,IN,1635413897,Hotel KNB Heritage
313,Turaif,31.6725,38.6637,79.05,27,0,6.91,SA,1635414033,Le Park Concord Turayf
314,Neuquén,-38.9516,-68.0591,66.09,34,0,6.91,AR,1635414034,Hotel el Prado
388,Fasa,28.9383,53.6482,76.24,11,0,0.00,IR,1635414056,Jahangardi Hotel
404,San Miguel de Tucumán,-26.8241,-65.2226,66.20,52,0,8.05,AR,1635414060,Hotel Embajador
429,Podgorica,42.4411,19.2636,73.51,34,0,1.14,ME,1635414069,Hilton Podgorica Crna Gora


In [22]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "Lng"]]

In [23]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))